In [ ]:
!pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot, cm
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score 
from sklearn import metrics

from statsmodels.stats.proportion import proportion_confint
from Functions.auc_delong_xu import auc_ci_Delong

In [ ]:
#read file with predictions from 5 runs
pred = pd.read_csv('~/deepensemble.csv', sep = ";")
pred

In [ ]:
cm = confusion_matrix(pred.y_test, pred.y_pred)
lower_acc, upper_acc = proportion_confint(sum(np.diag(cm)), np.sum(cm), 0.05, method = 'wilson')
ci_acc = np.array([lower_acc, upper_acc])

auc, auc_var, ci_auc = auc_ci_Delong(
    y_true=pred.y_test,
    y_scores=pred.y_prob_mean)

sensitivity = cm[1,1]/(cm[1,1]+cm[1,0])
lower_sens, upper_sens = proportion_confint(cm[1,1], (cm[1,1]+cm[1,0]), 0.05, method = 'wilson')
ci_sens = np.array([lower_sens, upper_sens])

specificity = cm[0,0]/(cm[0,0]+cm[0,1])
lower_spec, upper_spec = proportion_confint(cm[0,0], (cm[0,0]+cm[0,1]), 0.05, method = 'wilson')
ci_spec = np.array([lower_spec, upper_spec])

nll = log_loss(pred.y_test, pred.y_prob_mean)

print("Accuracy: ", round((np.mean(pred.y_pred == pred.y_test)),3), ' CI:', np.round(ci_acc,4) )
print("AUC: ", round(auc, 3), 'CI:', np.round(ci_auc, 3))
print("Sensitivity: ", round(sensitivity,3), 'CI:', np.round(ci_sens, 3))
print("Specificity: ", round(specificity,3), 'CI:', np.round(ci_spec, 3))
print("NLL: ", round(nll,4))
print("Confusion matrix: \n", 
      cm)
print("Normalized Confusion matrix: \n", 
      np.round(confusion_matrix(pred.y_test, pred.y_pred, normalize = 'true'),3))

In [ ]:
def ensembling_measures(df):
    print("Mittelwert: ", df.mean())
    print("Median: ", df.median())
    print("25% Quantile: ", df.quantile(0.25))
    print("75% Quantile: ", df.quantile(0.75))
    print("IQR Range: ", df.quantile(0.75) - df.quantile(0.25))

In [ ]:
correct_class = pred.loc[lambda pred: pred['y_test'] == pred['y_pred']]
print("Richtige Klasse -----------------------------------------------------------")
ensembling_measures(correct_class['prob_var'])

incorrect_class = pred.loc[lambda pred: pred['y_test'] != pred['y_pred']]
print("-----------------------------------------------------------")
print("Falsche Klasse")
ensembling_measures(incorrect_class['prob_var'])

correct_stroke = pred.loc[(pred['y_test']== pred['y_pred']) & (pred['y_test'] == 1)]
print("-----------------------------------------------------------")
print("Richtige Schlaganfall")
ensembling_measures(correct_stroke['prob_var'])

incorrect_stroke = pred.loc[(pred['y_test'] != pred['y_pred']) & (pred['y_test'] == 1)]
print("-----------------------------------------------------------")
print("Falsche Schlaganfall")
ensembling_measures(incorrect_stroke['prob_var'])

correct_tia = pred.loc[(pred['y_test']== pred['y_pred']) & (pred['y_test'] == 0)]
print("-----------------------------------------------------------")
print("Richtige TIA")
ensembling_measures(correct_tia['prob_var'])

incorrect_tia = pred.loc[(pred['y_test'] != pred['y_pred']) & (pred['y_test'] == 0)]
print("-----------------------------------------------------------")
print("Falsche TIA")
ensembling_measures(incorrect_tia['prob_var'])

In [ ]:
plt.hist(correct_class['prob_var'], bins = 20, color = "#2F5597")
plt.title("Varianz der Richtig-Klassifizierungen")
plt.xlabel("Varianz der Vorhersage-Wahrscheinlichkeiten")
plt.ylabel("Anzahl Vorhersagen")
plt.xlim(right = 0.16)
fig1 = plt.gcf()
plt.show()
p = '~/Histogramm_Varianzen_korrekte_Klasse.png'
fig1.savefig(p, dpi = 300,bbox_inches='tight')

In [ ]:
plt.hist(incorrect_class['prob_var'], bins = 20, color = '#7F7F7F')
plt.title("Varianz der Falsch-Klassifizierungen")
plt.xlabel("Varianz der Vorhersage-Wahrscheinlichkeiten")
plt.ylabel("Anzahl Vorhersagen")
plt.xlim(right = 0.16)
fig1 = plt.gcf()
plt.show()
p = '~/Histogramm_Varianzen_inkorrekte_Klasse.png'
fig1.savefig(p, dpi = 300,bbox_inches='tight')

In [ ]:
plt.hist(correct_stroke['prob_var'], bins = 20, color = "#2F5597")
plt.title("Varianz Richtig-Klassifizierungen der Klasse Schlaganfall")
plt.xlabel("Varianz der Vorhersage-Wahrscheinlichkeiten")
plt.ylabel("Anzahl Vorhersagen")
plt.xlim(right = 0.16)
fig1 = plt.gcf()
plt.show()
p = '~/Histogramm_Varianzen_korrekte_Stroke.png'
fig1.savefig(p, dpi = 300,bbox_inches='tight')

In [ ]:
plt.hist(incorrect_stroke['prob_var'], bins = 20, color = '#7F7F7F')
plt.title("Varianz Falsch-Klassifizierungen der Klasse Schlaganfall")
plt.xlabel("Varianz der Vorhersage-Wahrscheinlichkeiten")
plt.ylabel("Anzahl Vorhersagen")
plt.xlim(right = 0.16)
fig1 = plt.gcf()
plt.show()
p = '~/Histogramm_Varianzen_inkorrekte_Stroke.png'
fig1.savefig(p, dpi = 300,bbox_inches='tight')

In [ ]:
plt.hist(correct_tia['prob_var'], bins = 20, color = "#2F5597")
plt.title("Varianz Richtig-Klassifizierungen der Klasse TIA")
plt.xlabel("Varianz der Vorhersage-Wahrscheinlichkeiten")
plt.ylabel("Anzahl Vorhersagen")
plt.xlim(right = 0.16)
fig1 = plt.gcf()
plt.show()
p = '~/Histogramm_Varianzen_korrekte_TIA.png'
fig1.savefig(p, dpi = 300,bbox_inches='tight')

In [ ]:
plt.hist(incorrect_tia['prob_var'], bins = 20,  color = '#7F7F7F')
plt.title("Varianz Falsch-Klassifizierungen der Klasse TIA")
plt.xlabel("Varianz der Vorhersage-Wahrscheinlichkeiten")
plt.ylabel("Anzahl Vorhersagen")
plt.xlim(right = 0.16)
fig1 = plt.gcf()
plt.show()
p = '~/Histogramm_Varianzen_inkorrekt_TIA.png'
fig1.savefig(p, dpi = 300,bbox_inches='tight')